In [ ]:
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats import endpoints
import pandas as pd
import time

pg = players.find_players_by_full_name('paul george')[0]
pg_id = pg['id']

# get top 8 pg teammates with most mins played

clippers = teams.find_team_by_abbreviation('lac')
clippers_id = clippers['id']
clippers_roster_data = endpoints.commonteamroster.CommonTeamRoster(team_id=clippers_id, season = '2023-24').get_data_frames()[0]


player_minutes = []

# get the minutes for each player

for index, player in clippers_roster_data.iterrows():
    if player['PLAYER_ID'] != pg_id:
        player_id = player['PLAYER_ID']
        player_name = player['PLAYER']
        game_log = endpoints.playergamelog.PlayerGameLog(player_id=player_id, season='2023-24').get_data_frames()[0]
        total_minutes = game_log['MIN'].sum()
        player_minutes.append({'name': player_name, 'minutes': total_minutes, 'id': player_id})
        time.sleep(1)



# sort top 8

minutes_df = pd.DataFrame(player_minutes)
top_8 = minutes_df.sort_values('minutes', ascending=False).head(8)

top_8_id = []
for index, player in top_8.iterrows():
    top_8_id.append((player['id']))


# get the two pt and three pt for each player

top_8_shooting_splits = {}

for top_8_player_id in top_8_id:
    top_8_player_stats = endpoints.playerdashboardbygeneralsplits.PlayerDashboardByGeneralSplits(player_id=top_8_player_id, season='2023-24').get_data_frames()[0]
    three_pt_percent = top_8_player_stats['FG3_PCT'][0]
    two_pt_percent = round((top_8_player_stats['FGM'][0] - top_8_player_stats['FG3M'][0]) / (top_8_player_stats['FGA'][0] - top_8_player_stats['FG3A'][0]), 3)
    two_three_array = [two_pt_percent, three_pt_percent]
    top_8_shooting_splits[top_8_player_id] = two_three_array
    time.sleep(1)

    
print(top_8_shooting_splits)

    













In [3]:
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats import endpoints
import pandas as pd
import time

def get_top_8_teammates_shooting_splits(player_name, season):
    player = players.find_players_by_full_name(player_name)[0]
    player_id = player['id']

    # find team + roster
    team_abbreviation = find_player_team(player_id, season)
    team = teams.find_team_by_abbreviation(team_abbreviation)
    team_id = team['id']
    team_roster_data = endpoints.commonteamroster.CommonTeamRoster(team_id=team_id, season = season).get_data_frames()[0]
    time.sleep(1.5)

    # data analysis for top 8
    player_minutes = []
    for _, player in team_roster_data.iterrows():
        if player['PLAYER_ID'] != player_id:
            player_id = player['PLAYER_ID']
            player_name = player['PLAYER']
            game_log = endpoints.playergamelog.PlayerGameLog(player_id=player_id, season=season).get_data_frames()[0]
            total_minutes = game_log['MIN'].sum()
            player_minutes.append({'name': player_name, 'minutes': total_minutes, 'id': player_id})
            time.sleep(1.5)
    
    minutes_df = pd.DataFrame(player_minutes)
    top_8 = minutes_df.sort_values('minutes', ascending=False).head(8)
    
    top_8_id = []

    for _, player in top_8.iterrows():
        top_8_id.append((player['id']))

    # get shooting splits
    top_8_shooting_splits = {}
    
    for top_8_player_id in top_8_id:
        top_8_player_stats = endpoints.playerdashboardbygeneralsplits.PlayerDashboardByGeneralSplits(player_id=top_8_player_id, season=season).get_data_frames()[0]
        three_pt_percent = top_8_player_stats['FG3_PCT'][0]
        two_pt_percent = round((top_8_player_stats['FGM'][0] - top_8_player_stats['FG3M'][0]) / (top_8_player_stats['FGA'][0] - top_8_player_stats['FG3A'][0]), 3)
        two_three_array = [two_pt_percent, three_pt_percent]
        top_8_shooting_splits[top_8_player_id] = two_three_array
        time.sleep(1.5)
    
    return top_8_shooting_splits



def find_player_team(id, season): # helper func 1
    player_info = endpoints.playercareerstats.PlayerCareerStats(player_id=id, per_mode36='Totals').get_data_frames()[0]
    time.sleep(1.5)  
    
    season_info = player_info[player_info['SEASON_ID'] == season]
    
    if not season_info.empty:
        team_abbreviation = season_info['TEAM_ABBREVIATION'].iloc[0]
        return team_abbreviation
    else:
        return None
    

print(get_top_8_teammates_shooting_splits('trae young', '2020-21'))  

{1628989: [0.51, 0.363], 1629027: [0.491, 0.343], 203991: [0.594, 0.0], 1628381: [0.615, 0.399], 203524: [0.453, 0.321], 101150: [0.414, 0.399], 203992: [0.522, 0.438], 201568: [0.466, 0.406]}
